In [72]:
!pip install keras-tuner

In [73]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [74]:
fashion_mnist=keras.datasets.fashion_mnist

In [75]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [76]:
train_images=train_images/225.0
test_images=test_images/225.0


In [77]:
train_images[0].shape

(28, 28)

In [78]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [79]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernal', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernal', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [80]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [81]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:

tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 1 Complete [00h 08m 22s]
val_accuracy: 0.9171666502952576

Best val_accuracy So Far: 0.9171666502952576
Total elapsed time: 00h 08m 22s

Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |80                |48                
conv_1_kernal     |5                 |3                 
conv_2_filter     |48                |48                
conv_2_kernal     |3                 |3                 
dense_1_units     |64                |96                
learning_rate     |0.001             |0.001             

Epoch 1/3
 501/1688 [=======>......................] - ETA: 2:06 - loss: 0.5455 - accuracy: 0.8026

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()